Md. Tanvir Jawad <br>ID: 21101221

In [1]:
try:
    import pint
except ImportError:
    !pip install pint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 6.5 MB/s eta 0:00:00


In [2]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/modsim.py')

Downloaded modsim.py


In [3]:
from modsim import *

In [6]:
download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/data/World_population_estimates.html')

Downloaded World_population_estimates.html


In [7]:
from pandas import read_html

filename = 'World_population_estimates.html'
tables = read_html(filename, header=0, index_col=0, decimal='M')
table2 = tables[2]
table2.columns = ['census', 'prb', 'un', 'maddison',
                  'hyde', 'tanton', 'biraben', 'mj',
                  'thomlinson', 'durand', 'clark']

In [8]:
un = table2.un / 1e9
census = table2.census / 1e9

In [9]:
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'chap06.py')

Downloaded chap06.py


In [10]:
from chap06 import run_simulation

def plot_estimates():
    census.plot(style=':', label='US Census')
    un.plot(style='--', label='UN DESA')
    decorate(xlabel='Year',
             ylabel='World population (billions)')

In [16]:
from sympy import symbols

t = symbols('t')

In [17]:
expr = t + 1
expr

t + 1

In [18]:
expr.subs(t, 2)

3

In [19]:
from sympy import Function

f = Function('f')
f

f

In [20]:
f(t)

f(t)

In [21]:
from sympy import diff

dfdt = diff(f(t), t)
dfdt

Derivative(f(t), t)

In [22]:
from sympy import Eq

alpha = symbols('alpha')
eq1 = Eq(dfdt, alpha*f(t))
eq1

Eq(Derivative(f(t), t), alpha*f(t))

In [23]:
from sympy import dsolve

solution_eq = dsolve(eq1)
solution_eq

Eq(f(t), C1*exp(alpha*t))

In [24]:
C1 = symbols('C1')

In [25]:
particular = solution_eq.subs(C1, 1000)
particular

Eq(f(t), 1000*exp(alpha*t))

In [26]:
particular.subs(t, 0)

Eq(f(0), 1000)

In [28]:
r, K = symbols('r K')

In [29]:
eq2 = Eq(diff(f(t), t), r * f(t) * (1 - f(t)/K))
eq2

Eq(Derivative(f(t), t), r*(1 - f(t)/K)*f(t))

In [30]:
solution_eq = dsolve(eq2)
solution_eq

Eq(f(t), K*exp(C1*K + r*t)/(exp(C1*K + r*t) - 1))

In [31]:
general = solution_eq.rhs
general

K*exp(C1*K + r*t)/(exp(C1*K + r*t) - 1)

In [32]:
at_0 = general.subs(t, 0)
at_0

K*exp(C1*K)/(exp(C1*K) - 1)

In [33]:
from sympy import solve

p_0 = symbols('p_0')
solutions = solve(Eq(at_0, p_0), C1)

In [34]:
type(solutions), len(solutions)

(list, 1)

In [35]:
value_of_C1 = solutions[0]
value_of_C1

log(-p_0/(K - p_0))/K

In [36]:
particular = general.subs(C1, value_of_C1)
particular

-K*p_0*exp(r*t)/((K - p_0)*(-p_0*exp(r*t)/(K - p_0) - 1))

In [37]:
simpler = particular.simplify()
simpler

K*p_0*exp(r*t)/(K + p_0*exp(r*t) - p_0)

In [38]:
A = (K - p_0) / p_0
A

(K - p_0)/p_0

In [39]:
from sympy import exp

logistic = K / (1 + A * exp(-r*t))
logistic

K/(1 + (K - p_0)*exp(-r*t)/p_0)

In [40]:
(particular - logistic).simplify()

0

In [41]:
from sympy.printing.pycode import pycode

pycode(simpler)

'K*p_0*math.exp(r*t)/(K + p_0*math.exp(r*t) - p_0)'

In [42]:
alpha, beta = symbols('alpha beta')

In [43]:
eq3 = Eq(diff(f(t), t), alpha*f(t) + beta*f(t)**2)
eq3

Eq(Derivative(f(t), t), alpha*f(t) + beta*f(t)**2)

In [44]:
solution_eq = dsolve(eq3)
solution_eq

Eq(f(t), alpha*exp(alpha*(C1 + t))/(beta*(1 - exp(alpha*(C1 + t)))))

In [45]:
general = solution_eq.rhs
general

alpha*exp(alpha*(C1 + t))/(beta*(1 - exp(alpha*(C1 + t))))

In [46]:
at_0 = general.subs(t, 0)

In [47]:

solutions = solve(Eq(at_0, p_0), C1)
value_of_C1 = solutions[0]
value_of_C1

log(beta*p_0/(alpha + beta*p_0))/alpha

In [48]:
particular = general.subs(C1, value_of_C1)
particular.simplify()

alpha*p_0*exp(alpha*t)/(alpha - beta*p_0*exp(alpha*t) + beta*p_0)